# Frequency table
*By P. Stikker*<br>
https://PeterStatistics.com<br>
https://www.youtube.com/stikpet

In [1]:
import pandas as pd
import numpy as np

The example data I'm using is coming from an SPSS file. To get this in Python, I'm using the 'pyreadstat' library. See https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html for details

In [2]:
import pyreadstat

Now to get my data.

In [3]:
myDf, meta = pyreadstat.read_sav("FilmPreferences2.sav")

In [4]:
myDf.head()

,sex,age,educat,income,thriller,horror,comedy,adventur,docu,roman,munt,movies,tuschin,arena
0,0.0,25.0,2.0,2000.0,6.0,3.0,6.0,6.0,9.0,6.0,0.0,0.0,0.0,0.0
1,0.0,33.0,2.0,2350.0,4.0,2.0,7.0,8.0,4.0,7.0,0.0,0.0,0.0,1.0
2,0.0,44.0,3.0,4500.0,6.0,3.0,7.0,8.0,8.0,7.0,1.0,1.0,1.0,0.0
3,0.0,53.0,3.0,5800.0,4.0,1.0,5.0,9.0,7.0,4.0,1.0,1.0,1.0,0.0
4,0.0,64.0,2.0,4450.0,5.0,4.0,6.0,9.0,9.0,8.0,1.0,0.0,0.0,0.0


Lets start building a frequency table. First the counts of a column.

In [5]:
myFreqTable = myDf['educat'].value_counts()
myFreqTable

2.0    70
1.0    45
3.0    35
Name: educat, dtype: int64

This is actually a so-called 'dictionary'. I'll first convert this to a numpy array:

In [6]:
myFreqTable = np.array([myFreqTable.keys(),myFreqTable.values])
myFreqTable = np.swapaxes(myFreqTable,0,1)

In [7]:
myFreqTable

array([[ 2., 70.],
       [ 1., 45.],
       [ 3., 35.]])

Then into a panda dataframe, so it will look a bit nicer:

In [8]:
myFreqTable = pd.DataFrame(data=myFreqTable,columns=['educat', 'Frequency'])

In [9]:
myFreqTable

,educat,Frequency
0,2.0,70.0
1,1.0,45.0
2,3.0,35.0


The order seems a bit off. Lets fix that:

In [10]:
myFreqTable = myFreqTable.sort_values(by=['educat'])
myFreqTable = myFreqTable.reset_index(drop=True)
myFreqTable

,educat,Frequency
0,1.0,45.0
1,2.0,70.0
2,3.0,35.0


The frequencies are of course integers, so:

In [11]:
myFreqTable = myFreqTable.astype({'Frequency': 'int32'})
myFreqTable

,educat,Frequency
0,1.0,45
1,2.0,70
2,3.0,35


Add the percentages:

In [12]:
myFreqTable['Percent'] = myFreqTable['Frequency']/myFreqTable['Frequency'].sum()*100
myFreqTable

,educat,Frequency,Percent
0,1.0,45,30.000000
1,2.0,70,46.666667
2,3.0,35,23.333333


Add the cumulative percentages:

In [13]:
myFreqTable['Cumulative Percent'] = myFreqTable['Frequency'].cumsum() / myFreqTable['Frequency'].sum() * 100
myFreqTable

,educat,Frequency,Percent,Cumulative Percent
0,1.0,45,30.000000,30.000000
1,2.0,70,46.666667,76.666667
2,3.0,35,23.333333,100.000000


Yes, we could also do the 'cumsum()' directly on the Percent, but if those get rounded, then the cumulative percent will add the rounding errors. By doing the 'cumsum()' from the frequencies and then dividing by the sum of those, we don't need to worry about that.

The 'educat' were actually categories. These can be obtained from the meta data that we got when loading the SPSS file.

In [14]:
myDic = meta.variable_value_labels

In [15]:
myDic['educat']

{1.0: 'Lower level', 2.0: 'Intermediate level', 3.0: 'Higher level'}

Lets replace the categories with their text equivelants. The below code does seem to give an error, but it does actually work:

In [16]:
for i in range(len(myFreqTable['educat'])):
    myFreqTable['educat'][i] = myDic['educat'][myFreqTable['educat'][i]]

<ipython-input-16-f71cedc1c3a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myFreqTable['educat'][i] = myDic['educat'][myFreqTable['educat'][i]]
C:\Users\Peter\anaconda3\envs\Data Science - P37 build\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
myFreqTable

,educat,Frequency,Percent,Cumulative Percent
0,Lower level,45,30.000000,30.000000
1,Intermediate level,70,46.666667,76.666667
2,Higher level,35,23.333333,100.000000


Great.

I don't want to repeat these steps every time. So lets make a function out of it.

Although NOT recommended, I'm going to turn off the warnings. Just not to clutter this file too much

In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
def frequencyTable(data, column, aDictionary=None):
    # get the unique counts
    freqTable = data[column].value_counts()
    
    # convert the dictionary to a numpy array
    freqTableNP = np.array([freqTable.keys(),freqTable.values])
    freqTableNP = np.swapaxes(freqTableNP,0,1)
    
    # and the numpy array to a panda dataframe
    freqTable = pd.DataFrame(data=freqTableNP,columns=[column, 'Frequency'])
    
    # order the table
    freqTable = freqTable.sort_values(by=[column])
    freqTable = freqTable.reset_index(drop=True)
    
    # adjust frequency to int
    freqTable = freqTable.astype({'Frequency': 'int32'})
    
    # adding the percent and cumulative percent
    freqTable['Percent'] = freqTable['Frequency']/freqTable['Frequency'].sum()*100
    freqTable['Cumulative Percent'] = freqTable['Frequency'].cumsum() / freqTable['Frequency'].sum() * 100
    
    # if a dictionary is provided:
    if aDictionary!=None:
        for i in range(len(freqTable[column])):
            freqTable[column][i] = aDictionary[column][freqTable[column][i]]
    
    return freqTable

Lets run a few examples.

In [20]:
frequencyTable(myDf, 'educat', aDictionary=myDic)

,educat,Frequency,Percent,Cumulative Percent
0,Lower level,45,30.000000,30.000000
1,Intermediate level,70,46.666667,76.666667
2,Higher level,35,23.333333,100.000000


In [21]:
frequencyTable(myDf, 'educat')

,educat,Frequency,Percent,Cumulative Percent
0,1.0,45,30.000000,30.000000
1,2.0,70,46.666667,76.666667
2,3.0,35,23.333333,100.000000


In [22]:
frequencyTable(myDf, 'sex', aDictionary=myDic)

,sex,Frequency,Percent,Cumulative Percent
0,Female,70,46.666667,46.666667
1,Male,80,53.333333,100.000000


So far, so good. But....what if we have missing values?

In [23]:
myDf, meta = pyreadstat.read_sav("GSS2012-Aangepast.sav", user_missing=True)
myDic = meta.variable_value_labels
frequencyTable(myDf, 'mar1', aDictionary=myDic)

,mar1,Frequency,Percent,Cumulative Percent
0,MARRIED,972,49.240122,49.240122
1,WIDOWED,181,9.169200,58.409321
2,DIVORCED,314,15.906788,74.316109
3,SEPARATED,79,4.002026,78.318136
4,NEVER MARRIED,395,20.010132,98.328267
5,DK,22,1.114488,99.442756
6,NA,11,0.557244,100.000000


The 'DK' and 'NA' were actually missing values. We can get the information from the meta data:

In [24]:
myMissing = meta.missing_ranges
myMissing['mar1']

[{'lo': 0.0, 'hi': 0.0}, {'lo': 8.0, 'hi': 8.0}, {'lo': 9.0, 'hi': 9.0}]

Lets first create the frequency table, without the labels.

In [25]:
myDf2 = frequencyTable(myDf, 'mar1')

In [26]:
myDf2

,mar1,Frequency,Percent,Cumulative Percent
0,1.0,972,49.240122,49.240122
1,2.0,181,9.169200,58.409321
2,3.0,314,15.906788,74.316109
3,4.0,79,4.002026,78.318136
4,5.0,395,20.010132,98.328267
5,8.0,22,1.114488,99.442756
6,9.0,11,0.557244,100.000000


Now to loop through the missing values and determine the so called Valid Percent and Cumulative Valid Percent:

In [27]:
myDf2 = frequencyTable(myDf, 'mar1')

myMissing = meta.missing_ranges
myMissingAr = np.array(myMissing['mar1'])

isNonmisAr = np.array([])

# check if category is a missing category
for i in myDf2['mar1']:
    isNotMissing=1
    for j in myMissingAr:
        if i>=list(j.values())[0] and i<=list(j.values())[1]:
            isNotMissing=0
    isNonmisAr = np.append(isNonmisAr, [isNotMissing])

validSum = sum(myDf2['Frequency']*isNonmisAr)
validPerc = myDf2['Frequency']/validSum*isNonmisAr*100

# remove the valid percentage and cumulative percentage for the missing values categories
for i in range(len(validPerc)):
    if isNonmisAr[i]==0:
        validPerc[i]=np.nan

# add the new columns
myDf2['Valid Percent'] = validPerc

myDf2['Cumulative Valid Percent'] = myDf2['Valid Percent'].cumsum()

myDf2

,mar1,Frequency,Percent,Cumulative Percent,Valid Percent,Cumulative Valid Percent
0,1.0,972,49.240122,49.240122,50.077280,50.077280
1,2.0,181,9.169200,58.409321,9.325090,59.402370
2,3.0,314,15.906788,74.316109,16.177228,75.579598
3,4.0,79,4.002026,78.318136,4.070067,79.649665
4,5.0,395,20.010132,98.328267,20.350335,100.000000
5,8.0,22,1.114488,99.442756,NaN,NaN
6,9.0,11,0.557244,100.000000,NaN,NaN


I'm not going to do all this every time. Lets adjust the function.

In [28]:
def frequencyTable(data, column, aDictionary=None, missing=None):
    # get the unique counts
    freqTable = data[column].value_counts()
    
    # convert the dictionary to a numpy array
    freqTableNP = np.array([freqTable.keys(),freqTable.values])
    freqTableNP = np.swapaxes(freqTableNP,0,1)
    
    # and the numpy array to a panda dataframe
    freqTable = pd.DataFrame(data=freqTableNP,columns=[column, 'Frequency'])
    
    # order the table
    freqTable = freqTable.sort_values(by=[column])
    freqTable = freqTable.reset_index(drop=True)
    
    # adjust frequency to int
    freqTable = freqTable.astype({'Frequency': 'int32'})
    
    # adding the percent and cumulative percent
    freqTable['Percent'] = freqTable['Frequency']/freqTable['Frequency'].sum()*100
    freqTable['Cumulative Percent'] = freqTable['Frequency'].cumsum() / freqTable['Frequency'].sum() * 100
    
    # if a missing meta information is provided
    if missing!=None:
        myMissingAr = np.array(missing[column])
        isNonmisAr = np.array([])
        # check if category is a missing category
        for i in freqTable[column]:
            isNotMissing=1
            for j in myMissingAr:
                if i>=list(j.values())[0] and i<=list(j.values())[1]:
                    isNotMissing=0
            isNonmisAr = np.append(isNonmisAr, [isNotMissing])

        validSum = sum(freqTable['Frequency']*isNonmisAr)
        validPerc = freqTable['Frequency']/validSum*isNonmisAr*100

        # remove the valid percentage and cumulative percentage for the missing values categories
        for i in range(len(validPerc)):
            if isNonmisAr[i]==0:
                validPerc[i]=np.nan

        # add the new columns
        freqTable['Valid Percent'] = validPerc

        freqTable['Cumulative Valid Percent'] = freqTable['Valid Percent'].cumsum()

    # if a dictionary is provided:
    if aDictionary!=None:
        for i in range(len(freqTable[column])):
            freqTable[column][i] = aDictionary[column][freqTable[column][i]]
    
    return freqTable

In [29]:
frequencyTable(myDf, 'mar1', aDictionary=myDic, missing = meta.missing_ranges)

,mar1,Frequency,Percent,Cumulative Percent,Valid Percent,Cumulative Valid Percent
0,MARRIED,972,49.240122,49.240122,50.077280,50.077280
1,WIDOWED,181,9.169200,58.409321,9.325090,59.402370
2,DIVORCED,314,15.906788,74.316109,16.177228,75.579598
3,SEPARATED,79,4.002026,78.318136,4.070067,79.649665
4,NEVER MARRIED,395,20.010132,98.328267,20.350335,100.000000
5,DK,22,1.114488,99.442756,NaN,NaN
6,NA,11,0.557244,100.000000,NaN,NaN
